In [ ]:
import pandas as pd
import numpy as np

import os
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

import sys

import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display


from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, learning_curve, validation_curve
from sklearn.neighbors import KNeighborsClassifier as kNN
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier


from IPython.display import Audio

from utils.audio_preprocess import *
from utils.septr import SeparableTr
from utils.feature_exctraction_v2 import get_features

In [ ]:
from tqdm import tqdm

In [ ]:
import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Activation, Dense, Conv1D, Conv2D, MaxPooling1D,MaxPooling2D, Flatten, Dropout, BatchNormalization, Input
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

#import warnings
#if not sys.warnoptions:
#    warnings.simplefilter("ignore")
#warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
# # ave maria 🙏🏻🙏🏿
# from scikeras.wrappers import KerasClassifier

In [ ]:
Crema = "CREMA-D/AudioWAV/"
crema_directory_list = os.listdir(Crema)

file_emotion = []
file_path = []

NSIZE = len(crema_directory_list)
SAMPLE_SIZE = min(200, NSIZE)

In [ ]:
for file in crema_directory_list[:SAMPLE_SIZE]:

    file_path.append(Crema + file)

    part=file.split('_')
    if part[2] == 'SAD':
        file_emotion.append('sad')
    elif part[2] == 'ANG':
        file_emotion.append('angry')
    elif part[2] == 'DIS':
        file_emotion.append('disgust')
    elif part[2] == 'FEA':
        file_emotion.append('fear')
    elif part[2] == 'HAP':
        file_emotion.append('happy')
    elif part[2] == 'NEU':
        file_emotion.append('neutral')
    else:
        file_emotion.append('Unknown')
        
# dataframe 
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe Emotions/Path
path_df = pd.DataFrame(file_path, columns=['Path'])
Crema_df = pd.concat([emotion_df, path_df], axis=1)
Crema_df['Emotions'] = pd.Categorical(Crema_df['Emotions'])
Crema_df.head()

In [ ]:
path = np.array(Crema_df.Path[Crema_df.Emotions=='fear'])[1]
data, sampling_rate = librosa.load(path)

mel = librosa.feature.melspectrogram(y=data, sr=22050) 
mel.shape

In [ ]:
train, test = train_test_split(Crema_df, random_state=42, shuffle=True)

In [ ]:
len(test)

In [ ]:
x_train, y_train = [], []
x_test, y_test = [], []

# TRAIN + Augmentation
for path, emotion in tqdm(zip(train.Path, train.Emotions)):
    n_aug = 1 if "8" in path else 0
    feature = get_features(path, n_aug=n_aug, feats=False, debug=False)
    for ele in feature:
        x_train.append(ele)
        y_train.append(emotion)


In [ ]:
# TEST
for path, emotion in zip(test.Path, test.Emotions):
    feature = get_features(path, feats=False)
    for ele in feature:
        x_test.append(ele)
        y_test.append(emotion)

In [ ]:
max_rows, max_cols = 0,0
for atens in [x_train, x_test]:
    # Find the maximum number of rows and columns among all arrays
    max_rows = max(max_rows, max(a.shape[0] for a in atens))
    max_cols = max(max_cols, max(a.shape[1] for a in atens))

In [ ]:
for atens in [x_train, x_test]:
    for i, a in enumerate(atens):
        # Pad the smaller arrays with zeros to match the maximum shape
        atens[i] = np.pad(a, ((0, max_rows - a.shape[0]), (0, max_cols - a.shape[1])), 'constant')

# Stack the padded arrays using numpy.stack()
x_train = np.stack(x_train)
x_test = np.stack(x_test)


In [ ]:
# one hot encoding
encoder = OneHotEncoder()

y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()

In [ ]:
# making our data compatible to model.
x_train = np.expand_dims(x_train, axis=-1)
x_test =  np.expand_dims(x_test, axis=-1)

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
# scaling our data with sklearn's Standard scaler
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_test = scaler.transform(x_test)


### CNN

In [ ]:
def cnn_model(nfilters1=128, nfilters2=64, nfilters3=32, size_kernel=5, nstrides=1, size_pool=5):
    model=Sequential()
    model.add(Conv2D(filters=nfilters1, kernel_size=size_kernel, strides=nstrides, padding='same', activation='relu', input_shape=x_train.shape[1:]))
    model.add(Dropout(0.2))    
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv2D(filters=nfilters2, kernel_size=size_kernel, strides=nstrides, padding='same', activation='relu'))
    model.add(Dropout(0.15))
    model.add(BatchNormalization())    
    model.add(MaxPooling2D(pool_size=5, strides = 2, padding = 'same'))


    model.add(Conv2D(filters=nfilters3, kernel_size=size_kernel, strides=nstrides, padding='same', activation='relu'))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=5, strides = 2, padding = 'same'))

    #model.add(Conv1D(filters=nfilters4, kernel_size=size_kernel, strides=nstrides, padding='same', activation='relu'))
    #model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(units=16, activation='relu'))

    model.add(Dense(units=6, activation='softmax'))
    model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

    #rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.30, verbose=0, patience=2, min_lr=0.0000001)

    #model.fit(x_train, y_train, batch_size=32, epochs=25, validation_data=(x_test, y_test), callbacks=[rlrp])    
    
    #model.summary()

    return model

In [ ]:
cnn_model().fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=32)

### SepTr